In [ ]:
#  Notebook to load CSV into graph using DataFrame method
#
#  .  Data is the Neo4J Movie graph
#  .  Incidentally includes teaching methods relative to Dask DataFrame manipulation
#
#  .  I haven't fully checked all results below; there may be issues


In [ ]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")

In [ ]:
# ##################################################################
#
#  This section; basic graph and database setup, reset for test


In [18]:
#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")

--


In [ ]:
#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)

In [ ]:
#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)

In [19]:
#  CREATE A GRAPH

my_graph=my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, 2euC6kymsQ5R2G8KFxTWw6gfELkZV6enfgrkxT2NvAng, 0>


In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

# my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)

In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

In [ ]:
# ##################################################################
#
#  Load from source CSV, in this case we are using the Neo4J Movie graph


In [ ]:
import dask.dataframe as dd
import numpy as np

print("--")


In [20]:
#  Load a DataFrame from CSV, Nodes/Vertices

l_InputFile  = "./10_NMovieDB/24_nodes.txt"

df_all_nodes1 = dd.read_csv(
   l_InputFile,
   delimiter = ",",
   dtype = {
      # "id"        : int,
      "id"        : np.dtype('O'),
      "_labels"   : np.dtype('O'),
      # "born"      : float, 
      "born"      : np.dtype('O'),
      "name"      : np.dtype('O'),
      # "released"  : float,
      "released"  : np.dtype('O'),
      "tagline"   : np.dtype('O'),
      "title"     : np.dtype('O')
      })

print("--")


--


In [ ]:
#  Number of rows, two different ways

display(len(df_all_nodes1))
display(print("{} Rows".format(df_all_nodes1.shape[0].compute())))

#  Other output

display(df_all_nodes1.head(10))
display(df_all_nodes1[["born", "name"]].head(10))

In [ ]:
#  Print all rows, subsetted columns

for l_each in df_all_nodes1.iterrows():
   print(l_each[0], "   ", l_each[1]["_labels"], "   ", l_each[1]["name"])  


In [ ]:
# ##################################################################
#
#   https://www.askpython.com/python/examples/subset-a-dataframe
#   https://www.codegrepper.com/code-examples/python/convert+float+to+int+python+pandas
#   https://docs.dask.org/en/latest/generated/dask.dataframe.DataFrame.assign.html


In [ ]:
#  We have some issues we want to change with our data
#
#  .  Some of the property names have a leading underscore. Change those.
#  .  Some values which should be integer, are float.
#  .  The label values are currently ";Person" and ";Movie". Let's remove those semicolons.


In [21]:
#  Column renames

df_all_nodes2 = df_all_nodes1.rename(columns={"_id": "id", "_labels": "label"})

print("--")

--


In [23]:
#  Change float values to integer, remove leading semicolon from label

df_all_nodes3 = df_all_nodes2.assign(
   # born     = lambda x: x.born.fillna(0.0).astype(int), 
   born     = lambda x: x.born.fillna(0.0).astype(str), 
   # id       = lambda x: x.id.fillna(0.0).astype(int),
   id       = lambda x: x.id.fillna(0.0).astype(str),
   # released = lambda x: x.released.fillna(0.0).astype(int),
   released = lambda x: x.released.fillna(0.0).astype(str),
   label    = lambda x: x.label.astype(str).str[1:]
   )

print("--")

--


In [ ]:
display(len(df_all_nodes3))
display(df_all_nodes3.head(10))

In [ ]:
# ##################################################################

In [24]:
#  Now, populate Persons and Movies

df_persons = df_all_nodes3[df_all_nodes3["label"] == "Person"][["id", "label", "born", "name"]]

df_movies  = df_all_nodes3[df_all_nodes3["label"] == "Movie"][["id", "label", "released", "tagline", "title"]]

print("--")

--


In [ ]:

display(len(df_persons))
display(df_persons.head(10))
display(len(df_movies))
display(df_movies.head(10))


In [ ]:
# ##################################################################
#
#  Repeat the style of work from above, but now for edges
#
#  .  Some of the property names have a leading underscore. Change those.
#  .  Some values which should be integer, are float.


In [25]:
#  Load a DataFrame from CSV, Edges

l_InputFile  = "./10_NMovieDB/25_edges.txt"

df_all_edges1 = dd.read_csv(
   l_InputFile,
   delimiter = ",",
   dtype = {
      # "_start"    : float,
      "_start"    : np.dtype('O'),
      # "_end"      : float, 
      "_end"      : np.dtype('O'),
      "_type"     : np.dtype('O'),
      # "rating"    : float,
      "rating"    : np.dtype('O'),
      "roles"     : np.dtype('O'),
      "summary"   : np.dtype('O')
      })

print("--")

--


In [ ]:
display(len(df_all_edges1))
display(df_all_edges1.head(10))


In [26]:
df_all_edges2 = df_all_edges1.rename(columns={"_start": "START_ID", "_end": "END_ID", "_type": "TYPE",
   "rating": "RATING", "roles": "ROLES", "summary": "SUMMARY"})

df_all_edges3 = df_all_edges2.assign(
   # START_ID = lambda x: x.START_ID.fillna(0.0).astype(int), 
   START_ID = lambda x: x.START_ID.fillna(0.0).astype(str), 
   # END_ID   = lambda x: x.END_ID.fillna(0.0).astype(int),
   END_ID   = lambda x: x.END_ID.fillna(0.0).astype(str),
   # RATING   = lambda x: x.RATING.fillna(0.0).astype(int)
   RATING   = lambda x: x.RATING.fillna(0.0).astype(str)
   )

print("--")

--


In [ ]:
display(len(df_all_edges3))
display(df_all_edges3.head(10))

In [27]:
#  Split out the various edges

df_reviewed = df_all_edges3[df_all_edges3["TYPE"] == "REVIEWED"][["START_ID", "END_ID", "TYPE", "RATING", "SUMMARY"]]

df_wrote    = df_all_edges3[df_all_edges3["TYPE"] == "WROTE"   ][["START_ID", "END_ID", "TYPE"]]

df_produced = df_all_edges3[df_all_edges3["TYPE"] == "PRODUCED"][["START_ID", "END_ID", "TYPE"]]

df_directed = df_all_edges3[df_all_edges3["TYPE"] == "DIRECTED"][["START_ID", "END_ID", "TYPE"]]

df_follows  = df_all_edges3[df_all_edges3["TYPE"] == "FOLLOWS" ][["START_ID", "END_ID", "TYPE"]]

#  "roles" is a string similar to  '[ "a", "b", "c" ]'
#
#  This was automatically coming in as a list-
#  Cool

df_actedin  = df_all_edges3[df_all_edges3["TYPE"] == "ACTED_IN"][["START_ID", "END_ID", "TYPE", "ROLES"]]

print("--")

--


In [ ]:

display(len(df_reviewed))
display(df_reviewed.head(2))

display(len(df_wrote))
display(df_wrote.head(2))

display(len(df_produced))
display(df_produced.head(2))

display(len(df_directed))
display(df_directed.head(2))

display(len(df_follows))
display(df_follows.head(2))

display(len(df_actedin))
display(df_actedin.head(2))


In [ ]:
# ##################################################################

In [ ]:
#  Didn't need this; also don't know if it had any effect

# from dask.distributed import Client
# client = Client(n_workers=4, threads_per_worker=2)

# print("--")

In [28]:
# ##################################################################
#
#  Finally now, load the vertices/nodes into a graph
#
#  Some hinkiness we need to work around ..
#
#     .  The Dask DataFrames here were loaded from CSV, and those CSV
#        files were found, in scope.
#        The KG DataFrame importer will reference that same file
#        pathname, and the file will not be in scope. Basically,
#        it was expected that these files be on S3/GS all along.
#        I hate to have that dependency because, just one more thing
#        to have to manage.
#
#     .  So, we'll copy the DataFrames to Dask arrays, then back into
#        a Dask DataFrame.
#        Why not just copy the DaskDataFrame ?  Currently there is only 
#        shallow copies of DataFrames.
#
#  See,
#     https://stackoverflow.com/questions/52119342/how-do-i-convert-a-dask-dataframe-into-a-dask-array
#     https://docs.dask.org/en/latest/generated/dask.dataframe.from_dask_array.html


da_persons    = df_persons.to_dask_array()
da_movies     = df_movies.to_dask_array()
   #
da_directed   = df_directed.to_dask_array()
da_reviewed   = df_reviewed.to_dask_array()
da_wrote      = df_wrote.to_dask_array()
da_produced   = df_produced.to_dask_array()
da_follows    = df_follows.to_dask_array()
da_actedin    = df_actedin.to_dask_array()


df_persons2   = dd.io.from_dask_array(da_persons,  columns=["id", "label", "born", "name"]).compute()
df_movies2    = dd.io.from_dask_array(da_movies,   columns=["id", "label", "released", "tagline", "title"]).compute()
   #
df_directed2  = dd.io.from_dask_array(da_directed, columns=["START_ID", "END_ID", "TYPE"]).compute()
df_reviewed2  = dd.io.from_dask_array(da_reviewed, columns=["START_ID", "END_ID", "TYPE", "RATING", "SUMMARY"]).compute()
df_wrote2     = dd.io.from_dask_array(da_wrote, columns=["START_ID", "END_ID", "TYPE"]).compute()
df_produced2  = dd.io.from_dask_array(da_produced, columns=["START_ID", "END_ID", "TYPE"]).compute()
df_follows2   = dd.io.from_dask_array(da_follows, columns=["START_ID", "END_ID", "TYPE"]).compute()
df_actedin2   = dd.io.from_dask_array(da_actedin, columns=["START_ID", "END_ID", "TYPE", "ROLES"]).compute()

print("--")



--


In [29]:
from katana_enterprise.remote import import_data

print("--")


--


In [30]:

with import_data.DataFrameImporter(my_graph) as df_importer:
    
   # Person
   #
   df_importer.nodes_dataframe(
      df_persons2[["id", "label", "born", "name"]],
      id_column  = "id",
      id_space   = "Person"
      )
   #  Movie
   #
   df_importer.nodes_dataframe(
      df_movies2[["id", "label", "title", "tagline"]],
      id_column  = "id",
      id_space   = "Movie"
      )  
    
   #  DIRECTED
   #
   df_importer.edges_dataframe(
      df_directed2[["START_ID", "END_ID", "TYPE"]],
      source_id_space      = "Person",
      destination_id_space = "Movie",
      source_column        = "START_ID",
      destination_column   = "END_ID",
      type                 = "DIRECTED"
      )
   #  REVIEWED
   #
   df_importer.edges_dataframe(
      df_reviewed2[["START_ID", "END_ID", "TYPE", "RATING", "SUMMARY"]],
      source_id_space      = "Person",
      destination_id_space = "Movie",
      source_column        = "START_ID",
      destination_column   = "END_ID",
      type                 = "REVIEWED"
      )
   #  WROTE
   #
   df_importer.edges_dataframe(
      df_wrote2[["START_ID", "END_ID", "TYPE"]],
      source_id_space      = "Person",
      destination_id_space = "Movie",
      source_column        = "START_ID",
      destination_column   = "END_ID",
      type                 = "WROTE"
      )
   #  PRODUCED
   #
   df_importer.edges_dataframe(
      df_produced2[["START_ID", "END_ID", "TYPE"]],
      source_id_space      = "Person",
      destination_id_space = "Movie",
      source_column        = "START_ID",
      destination_column   = "END_ID",
      type                 = "PRODUCED"
      )
   #  FOLLOWS
   #
   df_importer.edges_dataframe(
      df_follows2[["START_ID", "END_ID", "TYPE"]],
      source_id_space      = "Person",
      destination_id_space = "Movie",
      source_column        = "START_ID",
      destination_column   = "END_ID",
      type                 = "FOLLOWS"
      )
   #  ACTEDIN
   #
   df_importer.edges_dataframe(
      df_actedin2[["START_ID", "END_ID", "TYPE", "ROLES"]],
      source_id_space      = "Person",
      destination_id_space = "Movie",
      source_column        = "START_ID",
      destination_column   = "END_ID",
      type                 = "ACTEDIN"
      )

   df_importer.execute()
    
    

          0/? [?op/s]

          0/? [?op/s]

          0/? [?op/s]

          0/? [?op/s]

In [ ]:

l_result1 = my_graph.query("""

   MATCH (a) 
   WITH DISTINCT LABELS(a) AS temp, COUNT(a) AS tempCnt
   UNWIND temp AS label
   RETURN label, SUM(tempCnt) AS cnt
   ORDER BY label
   
   """)

display(print(l_result1))


In [ ]:

l_result1 = my_graph.query("""

   MATCH (m)-[r]->(n) 
   WITH DISTINCT TYPE(r) AS temp, COUNT(r) AS tempCnt
   RETURN temp, tempCnt
   ORDER BY temp

   """)

display(print(l_result1))


In [ ]:

result = my_graph.query("""

   MATCH (x) -[r]-> (a)
   RETURN x, r AS rel, a
   
   """,
   contextualize=True)

result.view()


# Output a graph as a a local file

In [31]:

l_result1 = my_graph.query("""
   MATCH (n:Person) 
   """)
      #
display(print(l_result1))

l_result2 = my_graph.query("""
   MATCH (n) - [r: WROTE] -> (m)
   """)
      #
display(print(l_result2))


          0/? [?op/s]

OperationError: 5YPQygeez4PBcgtSfjKC9UghaeCmdq8fWTVH3GttktqL-3DqkGvjMhUGXpvdZo backtrace (QueryClient.cpp:637): unable to translate openCypher parse tree to internal IR (OpGraph.cpp:224): Syntax error: The query cannot be recognized by openCypher. (CypherOpGraphBuilder.cpp:70): TCK = SyntaxError:UnexpectedSyntax
Katana = SyntaxError:ParserError: TCK = SyntaxError:UnexpectedSyntax
Katana = SyntaxError:ParserError